In [1]:
import os
import random
import pandas as pd
from shutil import copyfile

# Set the path to your data directory
data_dir = r"/Users/hiteshwaralavala/Desktop/new exp/colored_images"
dest_dir = r"/Users/hiteshwaralavala/Desktop/new_Exp_!"

# Set the ratio for splitting data into train and test
train_ratio = 0.8

# Create train and test directories
train_dir = os.path.join(dest_dir, 'train')
test_dir = os.path.join(dest_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Create CSV files to store image names and labels
train_csv = os.path.join(dest_dir, 'train.csv')
test_csv = os.path.join(dest_dir, 'test.csv')

train_data = []
test_data = []

# Iterate through each label folder
for label in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, label)):
        images = os.listdir(os.path.join(data_dir, label))
        random.shuffle(images)
        num_train = int(len(images) * train_ratio)
        
        # Copy images to train folder and add to train CSV
        for img in images[:num_train]:
            src_path = os.path.join(data_dir, label, img)
            dst_path = os.path.join(train_dir, f"{label}_{img}")
            copyfile(src_path, dst_path)
            train_data.append((f"{label}_{img}", label))
        
        # Copy images to test folder and add to test CSV
        for img in images[num_train:]:
            src_path = os.path.join(data_dir, label, img)
            dst_path = os.path.join(test_dir, f"{label}_{img}")
            copyfile(src_path, dst_path)
            test_data.append((f"{label}_{img}", label))

# Write train and test CSV files
train_df = pd.DataFrame(train_data, columns=['Image', 'Label'])
train_df.to_csv(train_csv, index=False)

test_df = pd.DataFrame(test_data, columns=['Image', 'Label'])
test_df.to_csv(test_csv, index=False)

print("Data splitting and CSV creation complete.")
print("Train data: ",str(len(train_data)))
print("Test data: ",str(len(test_data)))


Data splitting and CSV creation complete.
Train data:  2929
Test data:  733


In [2]:
import os

# List of labels
labels = [ 'Mild', 'Moderate', 'No_DR', 'Proliferate_DR', 'Severe']

# Dictionary to store counts
image_counts = {}

# Iterate through each label folder and count images
for label in labels:
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        images = os.listdir(label_dir)
        image_counts[label] = len(images)
    else:
        image_counts[label] = 0

# Print counts for each label
for label, count in image_counts.items():
    print(f"{label}: {count} images")

# Calculate total count
total_count = sum(image_counts.values())
print(f"Total: {total_count} images")


Mild: 370 images
Moderate: 999 images
No_DR: 1805 images
Proliferate_DR: 295 images
Severe: 193 images
Total: 3662 images


In [9]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Reshape, Multiply, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Multiply
from tensorflow.keras import backend as K

def squeeze_excite_block(input_tensor, ratio=16):
    # Get the number of channels
    channels = int(input_tensor.shape[-1])

    # Reshape the input tensor to ensure it has the correct shape
    x = Reshape((1, 1, channels))(input_tensor)

    # Squeeze operation
    x = GlobalAveragePooling2D()(x)
    
    # Excitation operation
    x = Dense(channels // ratio, activation='relu')(x)
    x = Dense(channels, activation='sigmoid')(x)

    # Scale input features
    x = Multiply()([input_tensor, x])

    return x

# Define custom preprocessing function for Gaussian noise
def apply_gaussian_noise(image):
    noise = np.random.normal(loc=0, scale=0.1, size=image.shape)
    return image + noise

# Set paths
train_dir = "/Users/hiteshwaralavala/Desktop/new_exp_2/train"
train_csv = "/Users/hiteshwaralavala/Desktop/new_exp_2/train.csv"
image_size = (224, 224)  # DenseNet input size
num_classes = 5

# Iterate through each image in the train folder and resize
for filename in os.listdir(train_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        image_path = os.path.join(train_dir, filename)
        with Image.open(image_path) as img:
            img_resized = img.resize(image_size)
            img_resized.save(image_path)

# Read train CSV file
train_data = pd.read_csv(train_csv)

# Create ImageDataGenerator with Gaussian noise preprocessing and data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=apply_gaussian_noise,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Flow from directory with train_data
train_generator = datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=train_dir,
    x_col='Image',
    y_col='Label',
    target_size=image_size,
    batch_size=32,
    class_mode='categorical'
)

# Load DenseNet model without top layers
base_model = DenseNet121(weights='/Users/hiteshwaralavala/Desktop/new_exp_2/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers with SE block
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = squeeze_excite_block(x)  # Add SE block
x = Dense(1024, activation='relu')(x)

# Add Dropout regularization
x = Dropout(0.5)(x)

predictions = Dense(num_classes, activation='softmax')(x)

# Create final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers of DenseNet base
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(train_generator, epochs=20)

# Save model with .h5 extension
model.save('/Users/hiteshwaralavala/Desktop/new_exp_2/densnet_optimised_model_with_dropout_and_gaussian.h5')


Found 2929 validated image filenames belonging to 5 classes.
Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


92/92 ━━━━━━━━━━━━━━━━━━━━ 64s 631ms/step - accuracy: 0.5868 - loss: 1.1431
Epoch 2/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 60s 630ms/step - accuracy: 0.7330 - loss: 0.7576
Epoch 3/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 61s 636ms/step - accuracy: 0.7318 - loss: 0.6978
Epoch 4/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 61s 642ms/step - accuracy: 0.7376 - loss: 0.6947
Epoch 5/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 60s 634ms/step - accuracy: 0.7309 - loss: 0.6976
Epoch 6/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 61s 637ms/step - accuracy: 0.7546 - loss: 0.6526
Epoch 7/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 61s 642ms/step - accuracy: 0.7552 - loss: 0.6451
Epoch 8/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 61s 638ms/step - accuracy: 0.7747 - loss: 0.6049
Epoch 9/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 61s 636ms/step - accuracy: 0.7730 - loss: 0.5921
Epoch 10/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 60s 632ms/step - accuracy: 0.7660 - loss: 0.6144
Epoch 11/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 60s 630ms/step - accuracy: 0.7833 - loss: 0.5985
Epoch 12/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 60s 633ms/ste

In [10]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Set paths
test_dir = r"/Users/hiteshwaralavala/Desktop/new_exp_2/test"
test_csv = r"/Users/hiteshwaralavala/Desktop/new_exp_2/test.csv"
image_size = (224, 224)  # DenseNet input size
num_classes = 5

# Set the target size
target_size = (224, 224)

# Iterate through each image in the train folder
for filename in os.listdir(test_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        image_path = os.path.join(test_dir, filename)
        with Image.open(image_path) as img:
            img_resized = img.resize(target_size)
            img_resized.save(image_path)


# Read test CSV file
test_data = pd.read_csv(test_csv)

# Create ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

# Flow from directory with test_data
test_generator = datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=test_dir,
    x_col='Image',
    y_col='Label',
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Important to match predictions with actual labels
)

# Load trained DenseNet model
model = load_model('/Users/hiteshwaralavala/Desktop/new_exp_2/densnet_optimised_model_with_dropout_and_gaussian.h5')

# Evaluate model on test data
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)

# Get predicted labels
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)

# Convert actual labels to numeric
label_map = {label: i for i, label in enumerate(test_generator.class_indices)}
y_true = test_data['Label'].map(label_map).values

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1_score = 2 * (precision * recall) / (precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)

Found 733 validated image filenames belonging to 5 classes.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 16s 616ms/step - accuracy: 0.6230 - loss: 0.9477
Test Accuracy: 0.7776262164115906
23/23 ━━━━━━━━━━━━━━━━━━━━ 16s 658ms/step
Precision: 0.7611542838185447
Recall: 0.7776261937244202
F1 Score: 0.7693020768081917
